In [39]:
import pandas as pd
import numpy as np
from utils import reduce_memory_usage
from utils import break_datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_percentage_error,mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import joblib
import zipfile
import os
import category_encoders
from sklearn.preprocessing import StandardScaler
import joblib
import zipfile
import os


warnings.filterwarnings('ignore')

# Loading data

In [40]:
# Specify the ZIP file name
zip_filename = "../dataset/filtered.zip"

# Extract the model file from the ZIP archive
with zipfile.ZipFile(zip_filename, "r") as archive:
    # Extract the model file (named "your_model.pkl" in this example)
    archive.extract("filtered.pkl")
    
# Load the model
df = joblib.load("filtered.pkl")  # Replace with "pickle.load" if you used pickle

os.remove("filtered.pkl")


In [41]:
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,46,0,2016-01-01 00:00:00,53.2397,0,Retail,9045,2016.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,74,0,2016-01-01 00:00:00,43.0013,0,Parking,387638,1997.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,93,0,2016-01-01 00:00:00,52.4206,0,Office,33370,1982.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,105,0,2016-01-01 00:00:00,23.3036,1,Education,50623,NaN,5.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
4,106,0,2016-01-01 00:00:00,0.3746,1,Education,5374,NaN,4.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1


# Site 0 Measurement Correction: kBTU to kWh conversion

In [42]:
df[df['site_id']==0].meter_reading.describe()

count    561983.000000
mean        366.178624
std         422.144648
min           0.068300
25%          97.469600
50%         221.586000
75%         462.435000
max        4521.000000
Name: meter_reading, dtype: float64

In [43]:
df[df['site_id']==0].meter_reading *= 0.293014534

In [44]:
df['log_meter_reading']=np.log1p(df['meter_reading'])
df['log_square_feet']=np.log1p(df['square_feet'])
df= break_datetime(df)
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,sea_level_pressure,wind_direction,wind_speed,log_meter_reading,log_square_feet,year,weekofyear,dayofweek,month,hour
0,46,0,2016-01-01,53.2397,0,Retail,9045,2016.0,NaN,25.0,...,1019.7,0.0,0.0,3.993413,9.110078,2015,53,5,1,0
1,74,0,2016-01-01,43.0013,0,Parking,387638,1997.0,NaN,25.0,...,1019.7,0.0,0.0,3.784219,12.867830,2015,53,5,1,0
2,93,0,2016-01-01,52.4206,0,Office,33370,1982.0,NaN,25.0,...,1019.7,0.0,0.0,3.978196,10.415443,2015,53,5,1,0
3,105,0,2016-01-01,23.3036,1,Education,50623,NaN,5.0,3.8,...,1020.9,240.0,3.1,3.190624,10.832181,2015,53,5,1,0
4,106,0,2016-01-01,0.3746,1,Education,5374,NaN,4.0,3.8,...,1020.9,240.0,3.1,0.318163,8.589514,2015,53,5,1,0


# Missing Value Imputation

In [45]:
def percent_missing_val(df):

  percent_missing = (df.isnull().sum() * 100) / len(df)
  missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
  return missing_value_df

In [46]:
missing_value_df= percent_missing_val(df)


In [47]:
missing_value_df

,column_name,percent_missing
building_id,building_id,0.000000
meter,meter,0.000000
timestamp,timestamp,0.000000
meter_reading,meter_reading,0.000000
site_id,site_id,0.000000
primary_use,primary_use,0.000000
square_feet,square_feet,0.000000
year_built,year_built,54.890488
floor_count,floor_count,74.735249
air_temperature,air_temperature,0.398257


As the year built and floor count columns have more than 50% values are missing, so we will drop these two columns.As the year built and floor count columns have more than 50% values are missing, so we will drop these two columns.

In [48]:
df.drop(['year_built', 'floor_count'], axis=1,inplace=True)


For the weather features, fill the missing values with the daily mean value of the corresponding feature in the site

In [49]:
def nan_fillers(df):
  air_temp_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['air_temperature'].transform('mean')
  df['air_temperature'].fillna(air_temp_df, inplace=True)

  dew_temp_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['dew_temperature'].transform('mean')
  df['dew_temperature'].fillna(dew_temp_df, inplace=True)

  cloud_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['cloud_coverage'].transform('mean')
  df['cloud_coverage'].fillna(cloud_df, inplace=True)

  sea_level_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['sea_level_pressure'].transform('mean')
  df['sea_level_pressure'].fillna(sea_level_df, inplace=True)

  precip_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['precip_depth_1_hr'].transform('mean')
  df['precip_depth_1_hr'].fillna(precip_df, inplace=True)

  wind_dir_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['wind_direction'].transform('mean')
  df['wind_direction'].fillna(wind_dir_df, inplace=True)

  wind_speed_df=df.groupby(['site_id', 'dayofweek', 'weekofyear'])['wind_speed'].transform('mean')
  df['wind_speed'].fillna(wind_speed_df, inplace=True)


  return df

In [50]:
df= nan_fillers(df)

In [51]:
df.isnull().sum()

building_id                 0
meter                       0
timestamp                   0
meter_reading               0
site_id                     0
primary_use                 0
square_feet                 0
air_temperature             0
cloud_coverage         713295
dew_temperature             0
precip_depth_1_hr     2075909
sea_level_pressure     797538
wind_direction              0
wind_speed                  0
log_meter_reading           0
log_square_feet             0
year                        0
weekofyear                  0
dayofweek                   0
month                       0
hour                        0
dtype: int64

As there were lot of slices where all the values were NAN in 3 columns namely: cloud_coverage, precip_depth_1_hr and sea_level_pressure hence we will impute the rest of the nan values with the median value.

In [52]:
df['cloud_coverage'].fillna(df['cloud_coverage'].median(), inplace=True)
df['sea_level_pressure'].fillna(df['sea_level_pressure'].median(), inplace=True)
df['precip_depth_1_hr'].fillna(df['precip_depth_1_hr'].median(), inplace=True)

# Baseline Model: Decision Tree Regressor

For baseline model, we will use site id, primary use, square feet and air temperature as features and predict the consumption in hourly 
* The building id and site id are highly correlated, so we will use one of them
* The air temperature and dew temperature are highly correlated, so we will use one of them
* square feet has positive correalation with the meter readings, so we will use as a feature


### Train - Test Split

Split data so that first 10 months will be train set and last two months will be test set

In [53]:
df=df.sort_values(by='timestamp')
X_train, X_test= train_test_split(df, test_size=0.20, shuffle=False)

In [54]:
y_train = X_train['log_meter_reading']
y_test = X_test['log_meter_reading']
X_train.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)
X_test.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)

# Label Encoding

In [55]:
label_enc= LabelEncoder()
label_enc.fit(df['primary_use'])
X_train['primary_use']= label_enc.transform(X_train['primary_use'])
X_test['primary_use']= label_enc.transform(X_test['primary_use'])


In [56]:
X_train.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,log_square_feet,year,weekofyear,dayofweek,month,hour
0,46,0,2016-01-01,0,11,9045,25.0,6.0,20.0,-0.138889,1019.7,0.0,0.0,9.110078,2015,53,5,1,0
773,1036,0,2016-01-01,12,6,54207,1.9,4.0,-1.2,0.000000,1016.2,200.0,5.0,10.900584,2015,53,5,1,0
772,1035,0,2016-01-01,12,6,14585,1.9,4.0,-1.2,0.000000,1016.2,200.0,5.0,9.587817,2015,53,5,1,0
771,1034,0,2016-01-01,12,11,3907,1.9,4.0,-1.2,0.000000,1016.2,200.0,5.0,8.270781,2015,53,5,1,0
770,1033,0,2016-01-01,12,0,118489,1.9,4.0,-1.2,0.000000,1016.2,200.0,5.0,11.682584,2015,53,5,1,0


### Quantitative Analysis

Primary Use

In [57]:
features = ['primary_use','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)

MSE for train set is:  2.068112154551855
R2 for train set is:  0.09311555783117687
MAPE for train set is:  9.703686519582282
MAE for train set is:  1.129120306166245
MSE for test set is:  2.0900893280780233
R2 for test set is:  0.08470893112346256
MAPE for test set is:  9.170177779903103
MAE for test set is:  1.141752738083253


Square Feet

In [58]:
features = ['square_feet','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])

# Create an empty dictionary to store results
results_dict = {'train_mse': [], 'train_r2': [], 'train_mape': [], 'train_mae': [],
                'test_mse': [], 'test_r2': [], 'test_mape': [], 'test_mae': []}

train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)

# Store results in the dictionary

results_dict['train_mse'].append(train_error_mse)
results_dict['train_r2'].append(train_error_r2)
results_dict['train_mape'].append(train_error_mape)
results_dict['train_mae'].append(train_error_mae)
results_dict['test_mse'].append(test_error_mse)
results_dict['test_r2'].append(test_error_r2)
results_dict['test_mape'].append(test_error_mape)
results_dict['test_mae'].append(test_error_mae)


MSE for train set is:  0.4456042951306126
R2 for train set is:  0.804598797155116
MAPE for train set is:  0.3087783854899786
MAE for train set is:  0.4644514463752949
MSE for test set is:  0.5256841680636054
R2 for test set is:  0.7697926028258018
MAPE for test set is:  0.3199832621488208
MAE for test set is:  0.5071524318974513


In [59]:
# Convert the dictionary to a DataFrame for easy handling and visualization
DTresults_df = pd.DataFrame(results_dict)

# Save the DataFrame to a CSV file
DTresults_df.to_csv('./DT_results.csv', index=False)

Air Temperature

In [60]:
'''features = ['air_temperature','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)'''

'features = [\'air_temperature\',\'hour\',\'dayofweek\',\'month\']\nDTR = DecisionTreeRegressor(max_depth=13)\nDTR.fit(X_train[features],y_train)\ny_pred_train = DTR.predict(X_train[features])\ntrain_error_mse = mean_squared_error(y_train,y_pred_train)\ntrain_error_r2 = r2_score(y_train,y_pred_train)\ntrain_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)\ntrain_error_mae = mean_absolute_error(y_train,y_pred_train)\n\nprint("MSE for train set is: ",train_error_mse)\n\nprint("R2 for train set is: ",train_error_r2)\n\nprint("MAPE for train set is: ",train_error_mape)\n\nprint("MAE for train set is: ",train_error_mae)\n\ny_pred_test = DTR.predict(X_test[features])\ntest_error_mse = mean_squared_error(y_test,y_pred_test)\ntest_error_r2 = r2_score(y_test,y_pred_test)\ntest_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)\ntest_error_mae = mean_absolute_error(y_test,y_pred_test)\n\nprint("MSE for test set is: ",test_error_mse)\n\nprint("R2 for test set is: ",t

Site ID

In [61]:
'''features = ['site_id','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)'''

'features = [\'site_id\',\'hour\',\'dayofweek\',\'month\']\nDTR = DecisionTreeRegressor(max_depth=13)\nDTR.fit(X_train[features],y_train)\ny_pred_train = DTR.predict(X_train[features])\ntrain_error_mse = mean_squared_error(y_train,y_pred_train)\ntrain_error_r2 = r2_score(y_train,y_pred_train)\ntrain_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)\ntrain_error_mae = mean_absolute_error(y_train,y_pred_train)\n\nprint("MSE for train set is: ",train_error_mse)\n\nprint("R2 for train set is: ",train_error_r2)\n\nprint("MAPE for train set is: ",train_error_mape)\n\nprint("MAE for train set is: ",train_error_mae)\n\ny_pred_test = DTR.predict(X_test[features])\ntest_error_mse = mean_squared_error(y_test,y_pred_test)\ntest_error_r2 = r2_score(y_test,y_pred_test)\ntest_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)\ntest_error_mae = mean_absolute_error(y_test,y_pred_test)\n\nprint("MSE for test set is: ",test_error_mse)\n\nprint("R2 for test set is: ",test_erro

Square Feet & Site ID 

In [62]:
'''features = ['square_feet','site_id','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)'''

'features = [\'square_feet\',\'site_id\',\'hour\',\'dayofweek\',\'month\']\nDTR = DecisionTreeRegressor(max_depth=13)\nDTR.fit(X_train[features],y_train)\ny_pred_train = DTR.predict(X_train[features])\ntrain_error_mse = mean_squared_error(y_train,y_pred_train)\ntrain_error_r2 = r2_score(y_train,y_pred_train)\ntrain_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)\ntrain_error_mae = mean_absolute_error(y_train,y_pred_train)\n\nprint("MSE for train set is: ",train_error_mse)\n\nprint("R2 for train set is: ",train_error_r2)\n\nprint("MAPE for train set is: ",train_error_mape)\n\nprint("MAE for train set is: ",train_error_mae)\n\ny_pred_test = DTR.predict(X_test[features])\ntest_error_mse = mean_squared_error(y_test,y_pred_test)\ntest_error_r2 = r2_score(y_test,y_pred_test)\ntest_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)\ntest_error_mae = mean_absolute_error(y_test,y_pred_test)\n\nprint("MSE for test set is: ",test_error_mse)\n\nprint("R2 for test set

Square Feet & Site Id & Primary Use

In [63]:
'''features = ['square_feet','site_id','primary_use','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)'''

'features = [\'square_feet\',\'site_id\',\'primary_use\',\'hour\',\'dayofweek\',\'month\']\nDTR = DecisionTreeRegressor(max_depth=13)\nDTR.fit(X_train[features],y_train)\ny_pred_train = DTR.predict(X_train[features])\ntrain_error_mse = mean_squared_error(y_train,y_pred_train)\ntrain_error_r2 = r2_score(y_train,y_pred_train)\ntrain_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)\ntrain_error_mae = mean_absolute_error(y_train,y_pred_train)\n\nprint("MSE for train set is: ",train_error_mse)\n\nprint("R2 for train set is: ",train_error_r2)\n\nprint("MAPE for train set is: ",train_error_mape)\n\nprint("MAE for train set is: ",train_error_mae)\n\ny_pred_test = DTR.predict(X_test[features])\ntest_error_mse = mean_squared_error(y_test,y_pred_test)\ntest_error_r2 = r2_score(y_test,y_pred_test)\ntest_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)\ntest_error_mae = mean_absolute_error(y_test,y_pred_test)\n\nprint("MSE for test set is: ",test_error_mse)\n\nprint(

Square Feet & Site Id & Air Temperature

In [64]:
'''features = ['square_feet','site_id','air_temperature','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)'''

'features = [\'square_feet\',\'site_id\',\'air_temperature\',\'hour\',\'dayofweek\',\'month\']\nDTR = DecisionTreeRegressor(max_depth=13)\nDTR.fit(X_train[features],y_train)\ny_pred_train = DTR.predict(X_train[features])\ntrain_error_mse = mean_squared_error(y_train,y_pred_train)\ntrain_error_r2 = r2_score(y_train,y_pred_train)\ntrain_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)\ntrain_error_mae = mean_absolute_error(y_train,y_pred_train)\n\nprint("MSE for train set is: ",train_error_mse)\n\nprint("R2 for train set is: ",train_error_r2)\n\nprint("MAPE for train set is: ",train_error_mape)\n\nprint("MAE for train set is: ",train_error_mae)\n\ny_pred_test = DTR.predict(X_test[features])\ntest_error_mse = mean_squared_error(y_test,y_pred_test)\ntest_error_r2 = r2_score(y_test,y_pred_test)\ntest_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)\ntest_error_mae = mean_absolute_error(y_test,y_pred_test)\n\nprint("MSE for test set is: ",test_error_mse)\n\npr

All Features

In [65]:
features = ['site_id','air_temperature','log_square_feet','primary_use','hour','dayofweek','month']
DTR = DecisionTreeRegressor(max_depth=13)
DTR.fit(X_train[features],y_train)
y_pred_train = DTR.predict(X_train[features])

# Create an empty dictionary to store results
results_dict2 = {'train_mse': [], 'train_r2': [], 'train_mape': [], 'train_mae': [],
                'test_mse': [], 'test_r2': [], 'test_mape': [], 'test_mae': []}

train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

y_pred_test = DTR.predict(X_test[features])
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)
results_dict2['train_mse'].append(train_error_mse)
results_dict2['train_r2'].append(train_error_r2)
results_dict2['train_mape'].append(train_error_mape)
results_dict2['train_mae'].append(train_error_mae)
results_dict2['test_mse'].append(test_error_mse)
results_dict2['test_r2'].append(test_error_r2)
results_dict2['test_mape'].append(test_error_mape)
results_dict2['test_mae'].append(test_error_mae)

# Convert the dictionary to a DataFrame for easy handling and visualization
DTresults_df2 = pd.DataFrame(results_dict2)

# Save the DataFrame to a CSV file
DTresults_df2.to_csv('./DT_results2.csv', index=False)


MSE for train set is:  0.2526124311088502
R2 for train set is:  0.8892273404192134
MAPE for train set is:  0.19947479060257367
MAE for train set is:  0.33862862178285785
MSE for test set is:  0.32952735962519386
R2 for test set is:  0.8556935126343329
MAPE for test set is:  0.23986520373968187
MAE for test set is:  0.37963465382126155


# Baseline Model: Linear Regression

### Train - Test Split

In [66]:
df=df.sort_values(by='timestamp')
X_train, X_test= train_test_split(df, test_size=0.20, shuffle=False)
y_train = X_train['log_meter_reading']
y_test = X_test['log_meter_reading']
X_train.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)
X_test.drop(['meter_reading', 'log_meter_reading'], axis=1, inplace=True)
X_train = X_train[features]
X_test = X_test[features]

### One Hot Encoding

In [67]:
categorical_features = ["site_id","primary_use"]

In [68]:
X_train = pd.get_dummies(X_train, columns=categorical_features)
X_test = pd.get_dummies(X_test, columns=categorical_features )

In [69]:
X_train

,air_temperature,log_square_feet,hour,dayofweek,month,site_id_0,site_id_1,site_id_2,site_id_3,site_id_4,...,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,25.000000,9.110078,0,5,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,3.800000,11.309352,0,5,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,6.913043,11.704066,0,5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25.000000,12.867830,0,5,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,25.000000,10.415443,0,5,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224573,19.400000,8.871084,20,6,10,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9224586,19.400000,11.949288,20,6,10,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9224589,19.400000,12.631801,20,6,10,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
9224578,19.400000,10.496289,20,6,10,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [70]:
LR = LinearRegression()
LR.fit(X_train,y_train)

LinearRegression()

In [76]:
# Create an empty dictionary to store results
results_dict3 = {'train_mse': [], 'train_r2': [], 'train_mape': [], 'train_mae': [],
                'test_mse': [], 'test_r2': [], 'test_mape': [], 'test_mae': []}

y_pred_train = LR.predict(X_train)
train_error_mse = mean_squared_error(y_train,y_pred_train)
train_error_r2 = r2_score(y_train,y_pred_train)
train_error_mape = mean_absolute_percentage_error(y_train,y_pred_train)
train_error_mae = mean_absolute_error(y_train,y_pred_train)

print("MSE for train set is: ",train_error_mse)

print("R2 for train set is: ",train_error_r2)

print("MAPE for train set is: ",train_error_mape)

print("MAE for train set is: ",train_error_mae)

MSE for train set is:  0.7785723183887558
R2 for train set is:  0.6585879025326522
MAPE for train set is:  6.131151877496826
MAE for train set is:  0.6488162778052322


In [77]:
y_pred_test = LR.predict(X_test)
test_error_mse = mean_squared_error(y_test,y_pred_test)
test_error_r2 = r2_score(y_test,y_pred_test)
test_error_mape = mean_absolute_percentage_error(y_test,y_pred_test)
test_error_mae = mean_absolute_error(y_test,y_pred_test)

print("MSE for test set is: ",test_error_mse)

print("R2 for test set is: ",test_error_r2)

print("MAPE for test set is: ",test_error_mape)

print("MAE for test set is: ",test_error_mae)
results_dict3['train_mse'].append(train_error_mse)
results_dict3['train_r2'].append(train_error_r2)
results_dict3['train_mape'].append(train_error_mape)
results_dict3['train_mae'].append(train_error_mae)
results_dict3['test_mse'].append(test_error_mse)
results_dict3['test_r2'].append(test_error_r2)
results_dict3['test_mape'].append(test_error_mape)
results_dict3['test_mae'].append(test_error_mae)

# Convert the dictionary to a DataFrame for easy handling and visualization
LRresults_df3 = pd.DataFrame(results_dict3)

# Save the DataFrame to a CSV file
LRresults_df3.to_csv('./LR_results.csv', index=False)

MSE for test set is:  0.8133469909346982
R2 for test set is:  0.6438262620293005
MAPE for test set is:  5.8504288935961055
MAE for test set is:  0.6593858571208412


# Save the models

 Save Decision Tree Regressor

In [78]:
# Specify the zip file name
zip_filename = "../models/DTR_v0.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "temp_model.pkl"
    joblib.dump(DTR, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="DTR_v0.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)


Save Linear Regression

In [79]:
# Specify the zip file name
zip_filename = "../models/LR_v0.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "temp_model.pkl"
    joblib.dump(DTR, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="LR_v0.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)

Reuse the model by loading

In [80]:
# Specify the ZIP file name
zip_filename = "../models/LR_v0.zip"

# Extract the model file from the ZIP archive
with zipfile.ZipFile(zip_filename, "r") as archive:
    # Extract the model file (named "your_model.pkl" in this example)
    archive.extract("LR_v0.pkl")
    
# Load the model
model = joblib.load("LR_v0.pkl")  # Replace with "pickle.load" if you used pickle

os.remove("LR_v0.pkl")

# You can now use the "model" for predictions or other tasks
y_pred_test = LR.predict(X_test)
test_error = mean_squared_error(y_test,y_pred_test)
print("MSE for test set is: ",test_error)

MSE for test set is:  0.8133469909346982
